Create spark session

In [1]:
from pyspark import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext.getOrCreate()
spark = SparkSession(sc)

Load data set

In [29]:
df = spark.read.format("csv").option("header", "false").load('/Users/hyy/Desktop/Columbia/E 4571/Project/Final/ratings_Electronics.csv').cache()

Change index name of the data set

In [31]:
df = df.select(df._c0.alias("userId"), df._c1.alias("itemId"), df._c2.alias("rating"), df._c3.alias("timestamp"))
df.show(10)

+--------------+----------+------+----------+
|        userId|    itemId|rating| timestamp|
+--------------+----------+------+----------+
| AKM1MP6P0OYPR|0132793040|   5.0|1365811200|
|A2CX7LUOHB2NDG|0321732944|   5.0|1341100800|
|A2NWSAGRHCP8N5|0439886341|   1.0|1367193600|
|A2WNBOD3WNDNKT|0439886341|   3.0|1374451200|
|A1GI0U4ZRJA8WN|0439886341|   1.0|1334707200|
|A1QGNMC6O1VW39|0511189877|   5.0|1397433600|
|A3J3BRHTDRFJ2G|0511189877|   2.0|1397433600|
|A2TY0BTJOTENPG|0511189877|   5.0|1395878400|
|A34ATBPOK6HCHY|0511189877|   5.0|1395532800|
| A89DO69P0XZ27|0511189877|   5.0|1395446400|
+--------------+----------+------+----------+
only showing top 10 rows



Function used to control sparsity of the data set

In [32]:
def calculate_sparsity(itemlimit, userlimit, df):
    product = df.groupBy("itemId").count()
    product_filter = product.filter(product['count'] > itemlimit)
    Data = df.join(product_filter, ['itemId'], 'leftsemi')
    
    user = Data.groupBy("userId").count()
    user_filter = user.filter(user['count'] > userlimit)
    DF = Data.join(user_filter, ['userId'], 'leftsemi')
    
    available = DF.count()
    product_total = DF.select("itemId").distinct().count()
    user_total = DF.select("userId").distinct().count()
    
    print("available rating: " + str(available))
    print("distinct product: " + str(product_total))
    print("distinct user: " + str(user_total))
    
    result = (float(available)/(float(product_total) * float(user_total)))*100
    print(result)
    
    return DF

We choose the subset that user rated more twice and product was rated more than twice

In [33]:
from pyspark.sql.functions import udf, col, regexp_replace
df=calculate_sparsity(2,2,df).cache()

available rating: 3374805
distinct product: 208371
distinct user: 615996
0.002629259887833436


Change the product id into int value

In [34]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer
stringIndexer = StringIndexer(inputCol="itemId", outputCol="ProductIndex")
user = stringIndexer.fit(df)
indexed = user.transform(df).cache()

Change the user id into int value

In [35]:
from pyspark.sql.types import *
indexed_distinct=indexed.select("userId").distinct()
User_id = spark.createDataFrame(indexed_distinct.rdd.map(lambda x: x[0]).zipWithIndex(), StructType([StructField("userId", StringType(), True),StructField("User_ID", IntegerType(), True)])).cache()

In [36]:
joined=indexed.join(User_id,indexed.userId==User_id.userId).cache()

Data cleaning process

In [37]:
def inte(f):
    return int(f)

inte_udf = udf(inte)

rating_data = joined.withColumn('Product_ID', inte_udf(col("ProductIndex")))
rating_data = rating_data.withColumn("Product_ID", rating_data["Product_ID"].cast(IntegerType()))
rating_data = rating_data.withColumn("rating", rating_data["rating"].cast(FloatType()))
rating_cleaned = rating_data.select('User_ID', 'Product_ID', 'rating')

In [38]:
rating_cleaned.show(10)

+-------+----------+------+
|User_ID|Product_ID|rating|
+-------+----------+------+
|      0|     13093|   4.0|
|      0|     10620|   5.0|
|      0|        53|   5.0|
|      1|      2208|   5.0|
|      1|     38247|   5.0|
|      1|      1857|   5.0|
|      1|     45751|   4.0|
|      1|     33783|   2.0|
|      1|      5416|   5.0|
|      2|     14873|   5.0|
+-------+----------+------+
only showing top 10 rows



Split the data set into train, validation and test set with 60%,20%,20% respectively 

In [39]:
train_rdd, valid_rdd, test_rdd = rating_cleaned.rdd.randomSplit(weights=[6, 2, 2], seed=0)
train_df = train_rdd.toDF()
valid_df = valid_rdd.toDF()
test_df = test_rdd.toDF().cache()

In the baseline model, we use both train set and validation set as training data

In [40]:
train_valid_df=train_df.union(valid_df).cache()

Calculate the rating mean of the whole training data

In [41]:
mean=train_valid_df.select('rating').groupBy().avg("rating").take(1)[0][0]
mean

4.159511466811338

Calculate the user bias

In [44]:
user_bias=train_valid_df.groupBy("User_ID").avg('rating')
user_bias=user_bias.select('User_ID',(((user_bias['avg(rating)'])-mean).alias("user_bias"))).cache()
user_bias.orderBy("User_ID").show(10)

+-------+--------------------+
|User_ID|           user_bias|
+-------+--------------------+
|      0|  0.5071551998553288|
|      1|  0.8404885331886618|
|      2| -0.4928448001446717|
|      3|  0.8404885331886618|
|      4|  0.8404885331886618|
|      5|  0.8404885331886618|
|      6|-0.32617813347800473|
|      7| 0.09048853318866179|
|      8| -0.6595114668113382|
|      9|  0.8404885331886618|
+-------+--------------------+
only showing top 10 rows



Calculate the product bias

In [46]:
item_bias=train_valid_df.groupBy("Product_ID").avg('rating')
item_bias=item_bias.select('Product_ID',(((item_bias['avg(rating)'])-mean).alias("item_bias"))).cache()
item_bias.orderBy("Product_ID").show(10)

+----------+-------------------+
|Product_ID|          item_bias|
+----------+-------------------+
|         0|0.40516743227123087|
|         1|-0.1884565409177026|
|         2| 0.6097337101880997|
|         3| 0.6250537948461652|
|         4| 0.4643045007123421|
|         5|0.23327605755513314|
|         6| 0.2898029308380252|
|         7|0.47174355058229267|
|         8|0.42012370879443406|
|         9| 0.2307535101471867|
+----------+-------------------+
only showing top 10 rows



Find the top 20 popular products according to the product bias

In [65]:
popular_item=item_bias.orderBy('item_bias',ascending=False).select('Product_ID').take(20)
popular_item_id=[]
for i in range(len(popular_item)):
    popular_item_id.append(popular_item[i][0])
popular_item_id

[128593,
 167071,
 93983,
 114290,
 143820,
 70175,
 78309,
 70528,
 113000,
 98225,
 153851,
 69795,
 146193,
 160204,
 74848,
 99084,
 118587,
 158828,
 175993,
 119492]

In [66]:
from pyspark.sql.functions import col, udf, array, collect_list
train_groupby_item = train_df.groupBy("User_ID").agg(collect_list("Product_ID")).orderBy('User_ID').cache()
train_list=train_groupby_item.take(train_groupby_item.count())
train_list

[Row(User_ID=0, collect_list(Product_ID)=[13093, 10620]),
 Row(User_ID=1, collect_list(Product_ID)=[38247]),
 Row(User_ID=2, collect_list(Product_ID)=[14873]),
 Row(User_ID=3, collect_list(Product_ID)=[20443, 17759, 264]),
 Row(User_ID=5, collect_list(Product_ID)=[26388, 30]),
 Row(User_ID=6, collect_list(Product_ID)=[29881, 30026, 4215, 102304, 48013, 6807, 164028, 107023, 8387, 4411]),
 Row(User_ID=7, collect_list(Product_ID)=[37380, 2645, 742, 195645]),
 Row(User_ID=8, collect_list(Product_ID)=[52041, 40208]),
 Row(User_ID=9, collect_list(Product_ID)=[14144]),
 Row(User_ID=10, collect_list(Product_ID)=[59827, 85, 37341]),
 Row(User_ID=11, collect_list(Product_ID)=[144341, 5497, 112311]),
 Row(User_ID=12, collect_list(Product_ID)=[394, 64758, 114677]),
 Row(User_ID=13, collect_list(Product_ID)=[24250, 1158]),
 Row(User_ID=14, collect_list(Product_ID)=[1917]),
 Row(User_ID=15, collect_list(Product_ID)=[138885, 27926]),
 Row(User_ID=16, collect_list(Product_ID)=[24366]),
 Row(User_ID=1

Filter out the users in the test set but not in the train set

In [67]:
user_in_train=train_df.select('User_ID').distinct().cache()
item_in_train=train_df.select('Product_ID').distinct().cache()
filtered_test_df=test_df.join(user_in_train,(test_df.User_ID==user_in_train.User_ID)).select(test_df.User_ID,'Product_ID','rating')
filtered_test_df=filtered_test_df.join(item_in_train,(filtered_test_df.Product_ID==item_in_train.Product_ID)).select('User_ID',filtered_test_df.Product_ID).cache()
filtered_test_df.orderBy('User_ID').show()

+-------+----------+
|User_ID|Product_ID|
+-------+----------+
|      1|     33783|
|      1|     45751|
|      1|      5416|
|      1|      1857|
|      2|      2047|
|      5|       171|
|      6|    102555|
|      6|     12887|
|      6|    104722|
|      6|      7392|
|      8|     44965|
|      9|      2507|
|     10|     46288|
|     13|      6897|
|     14|        14|
|     14|      1544|
|     14|       175|
|     14|     85069|
|     16|     21189|
|     16|     10846|
+-------+----------+
only showing top 20 rows



In [68]:
test_groupby_item = filtered_test_df.groupBy("User_ID").agg(collect_list("Product_ID")).orderBy('User_ID').cache()
test_groupby_item.show()

+-------+------------------------+
|User_ID|collect_list(Product_ID)|
+-------+------------------------+
|      1|    [5416, 45751, 185...|
|      2|                  [2047]|
|      5|                   [171]|
|      6|    [7392, 102555, 10...|
|      8|                 [44965]|
|      9|                  [2507]|
|     10|                 [46288]|
|     13|                  [6897]|
|     14|    [85069, 1544, 175...|
|     16|    [6, 5, 21189, 10846]|
|     17|    [18242, 945, 9748...|
|     18|           [46685, 5094]|
|     19|                  [6944]|
|     20|           [1845, 68765]|
|     21|                  [7735]|
|     22|                  [4398]|
|     25|                  [1162]|
|     28|              [384, 519]|
|     30|                [127532]|
|     33|                  [1933]|
+-------+------------------------+
only showing top 20 rows



Function used to find 10 recommend products for each user

In [78]:
def recommend(*param):
    global popular_item_id
    user_item=[param[0][0]]
    value=[]
    count=0
    for i in range(20):
        if popular_item_id[i] not in param[0][1]:
            value.append(popular_item_id[i])
            count+=1
            if count==10:
                break
    user_item.append(value)
    return user_item

In [70]:
recommend_list=[]
for i in range(len(train_list)):
    recommend_list.append(recommend(train_list[i]))

Prediction of the recommendations for each user

In [79]:
train_recommend=sc.parallelize(recommend_list).toDF().select(col('_1').alias('User_ID'),col('_2').alias('Recommend_Item_train')).cache()
train_recommend.show(10)

+-------+--------------------+
|User_ID|Recommend_Item_train|
+-------+--------------------+
|      0|[128593, 167071, ...|
|      1|[128593, 167071, ...|
|      2|[128593, 167071, ...|
|      3|[128593, 167071, ...|
|      5|[128593, 167071, ...|
|      6|[128593, 167071, ...|
|      7|[128593, 167071, ...|
|      8|[128593, 167071, ...|
|      9|[128593, 167071, ...|
|     10|[128593, 167071, ...|
+-------+--------------------+
only showing top 10 rows



Truly purchase in the test set

In [80]:
test_recommend=test_groupby_item.select(col('User_ID'),col('collect_list(Product_ID)').alias('Recommend_Item_test')).cache()
test_recommend.show(10)

+-------+--------------------+
|User_ID| Recommend_Item_test|
+-------+--------------------+
|      1|[5416, 45751, 185...|
|      2|              [2047]|
|      5|               [171]|
|      6|[7392, 102555, 10...|
|      8|             [44965]|
|      9|              [2507]|
|     10|             [46288]|
|     13|              [6897]|
|     14|[85069, 1544, 175...|
|     16|[6, 5, 21189, 10846]|
+-------+--------------------+
only showing top 10 rows



In [81]:
final_test=test_recommend.join(train_recommend,(test_recommend.User_ID==train_recommend.User_ID)).select(test_recommend.User_ID,'Recommend_Item_test','Recommend_Item_train').orderBy('User_ID').cache()
final_test.show(10)

+-------+--------------------+--------------------+
|User_ID| Recommend_Item_test|Recommend_Item_train|
+-------+--------------------+--------------------+
|      1|[5416, 45751, 185...|[128593, 167071, ...|
|      2|              [2047]|[128593, 167071, ...|
|      5|               [171]|[128593, 167071, ...|
|      6|[7392, 102555, 10...|[128593, 167071, ...|
|      8|             [44965]|[128593, 167071, ...|
|      9|              [2507]|[128593, 167071, ...|
|     10|             [46288]|[128593, 167071, ...|
|     13|              [6897]|[128593, 167071, ...|
|     14|[85069, 1544, 175...|[128593, 167071, ...|
|     16|[6, 5, 21189, 10846]|[128593, 167071, ...|
+-------+--------------------+--------------------+
only showing top 10 rows



In [74]:
final_test_list=final_test.take(final_test.count())
final_test_list

[Row(User_ID=1, Recommend_Item_test=[5416, 45751, 1857, 33783], Recommend_Item_train=[128593, 167071, 93983, 114290, 143820, 70175, 78309, 70528, 113000, 98225]),
 Row(User_ID=2, Recommend_Item_test=[2047], Recommend_Item_train=[128593, 167071, 93983, 114290, 143820, 70175, 78309, 70528, 113000, 98225]),
 Row(User_ID=5, Recommend_Item_test=[171], Recommend_Item_train=[128593, 167071, 93983, 114290, 143820, 70175, 78309, 70528, 113000, 98225]),
 Row(User_ID=6, Recommend_Item_test=[7392, 102555, 104722, 12887], Recommend_Item_train=[128593, 167071, 93983, 114290, 143820, 70175, 78309, 70528, 113000, 98225]),
 Row(User_ID=8, Recommend_Item_test=[44965], Recommend_Item_train=[128593, 167071, 93983, 114290, 143820, 70175, 78309, 70528, 113000, 98225]),
 Row(User_ID=9, Recommend_Item_test=[2507], Recommend_Item_train=[128593, 167071, 93983, 114290, 143820, 70175, 78309, 70528, 113000, 98225]),
 Row(User_ID=10, Recommend_Item_test=[46288], Recommend_Item_train=[128593, 167071, 93983, 114290, 

Count the true positive count of the baseline model

In [75]:
def recommend_count(*param):
    global final_test_list
    user_item=[param[0][0]]
    count=len(set(param[0][1])&set(param[0][2]))
    user_item.append(count)
    return user_item

In [76]:
recommend_num=[]
for i in range(len(final_test_list)):
    recommend_num.append(recommend_count(final_test_list[i]))

In [77]:
count=0
for i in range(len(recommend_num)):
    if recommend_num[i][1]!=0:
        count+=1
print(count)

9
